# Environment Preparation

## Imports

In [1]:
import re
import os
import io
import boto3
import pickle
import numpy             as np
import pandas            as pd
import seaborn           as sns
import datetime          as dt
import umap.umap_        as umap

from matplotlib               import pyplot as plt
from tabulate                 import tabulate

from sklearn                  import preprocessing as pp
from sklearn                  import metrics as m
from scipy.cluster            import hierarchy as hc

from sqlalchemy               import create_engine
from sqlalchemy.pool          import NullPool
from dotenv                   import load_dotenv

# Data Collection

In [2]:
#list my buckets, its files and load a file from S3 AWS:
AWS_REGION = "us-east-1"
S3_BUCKET_NAME = "loyals-dataset"

#buckets
client = boto3.client("s3", region_name=AWS_REGION)
response = client.list_buckets()

print("Listing Amazon S3 Buckets:")
for bucket in response['Buckets']:
    print(f"-- {bucket['Name']}")

#files
s3_resource = boto3.resource("s3", region_name=AWS_REGION)
s3_bucket = s3_resource.Bucket(S3_BUCKET_NAME)         
print('Listing Amazon S3 Bucket objects/files:')

for obj in s3_bucket.objects.all():
    print(f'-- {obj.key}')   
          
#load files
df_raw = pd.read_csv(io.BytesIO(obj.get()['Body'].read()), encoding='iso-8859-1') 

Listing Amazon S3 Buckets:
-- loyals-dataset
Listing Amazon S3 Bucket objects/files:


-- Ecommerce.csv


In [3]:
# # read local data
# path = '/Users/home/repos/pa005_fidelity_program/'
# df_raw = pd.read_csv(path +'data/raw/Ecommerce.csv', encoding='unicode_escape' )#, #encoding='iso-8859-1'

In [4]:
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Unnamed: 8
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom,NaN
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom,NaN
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN


In [5]:
df_raw = df_raw.drop('Unnamed: 8', axis=1).copy()
df_raw.sample(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
40292,539735,22625,RED KITCHEN SCALES,1,19-Dec-16,16.98,NaN,United Kingdom
519657,580160,22551,PLASTERS IN TIN SPACEBOY,12,30-Nov-17,1.65,12700.0,France
500218,578746,23168,CLASSIC CAFE SUGAR DISPENSER,10,23-Nov-17,3.29,NaN,United Kingdom


# Data Description

In [6]:
df1 = df_raw.copy()

## Rename Columns

In [7]:
df1.sample(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
89453,543909,22722,SET OF 6 SPICE TINS PANTRY DESIGN,6,12-Feb-17,8.29,NaN,United Kingdom
74055,542399,21355,TOAST ITS - I LOVE YOU,12,25-Jan-17,1.25,15549.0,United Kingdom
76861,542664,22030,SWALLOWS GREETING CARD,3,29-Jan-17,0.83,NaN,United Kingdom


In [8]:
df1.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [9]:
df1.columns = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country']

## Feature Description 

In [10]:
# Explain feature meanings
tab_meanings = [['Columns', 'Meaning'],
        ['invoice_no', 'unique identifier of each transaction'],
        ['stock_code', 'item code'],
        ['description', 'item name'],
        ['quantity', 'quantity of each item purchased per transaction'],
        ['invoice_date', 'the day the transaction took place'],
        ['unit_price', 'product price per unit'],
        ['customer_id', 'unique customer identifier'],
        ['country', 'customer\'s country of residence']
      ]
print(tabulate(tab_meanings, headers='firstrow', stralign='left', tablefmt='simple'))

Columns       Meaning
------------  -----------------------------------------------
invoice_no    unique identifier of each transaction
stock_code    item code
description   item name
quantity      quantity of each item purchased per transaction
invoice_date  the day the transaction took place
unit_price    product price per unit
customer_id   unique customer identifier
country       customer's country of residence


In [11]:
df1.sample(3)

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
171152,551391,22727,ALARM CLOCK BAKELIKE RED,21,26-Apr-17,3.75,15514.0,United Kingdom
69652,541975,22629,SPACEBOY LUNCH BOX,12,22-Jan-17,1.95,NaN,Hong Kong
94150,544332,22241,GARLAND WOODEN HAPPY EASTER,12,15-Feb-17,1.25,17419.0,United Kingdom


## Data Dimensions

In [12]:
print(f'Number of rows: {df1.shape[0]}')
print(f'Number of columns: {df1.shape[1]}')

Number of rows: 541909
Number of columns: 8


In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice_no    541909 non-null  object 
 1   stock_code    541909 non-null  object 
 2   description   540455 non-null  object 
 3   quantity      541909 non-null  int64  
 4   invoice_date  541909 non-null  object 
 5   unit_price    541909 non-null  float64
 6   customer_id   406829 non-null  float64
 7   country       541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


## Check NA

In [14]:
df1.isna().sum()

## Replace NA

In [ ]:
#separate dataset
df_missing = df1.loc[df1['customer_id'].isna()]
df_not_missing = df1.loc[~df1['customer_id'].isna()]

#create reference
df_invoice = pd.DataFrame(df_missing['invoice_no'].drop_duplicates() )
df_invoice['customer_id'] = np.arange(19000,19000+len(df_invoice),1)

# merge original with reference dataframe
df1 = pd.merge(df1, df_invoice, on='invoice_no', how='left')

#coalesce equivalent
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

# drop extra columns
df1 = df1.drop(columns = ['customer_id_x','customer_id_y'], axis=1 )

In [ ]:
#check NA
df1.isna().sum()

## Change Types

In [ ]:
#correct data types ensure correct calculations using the columns on next sessions

In [ ]:
df1.dtypes

In [ ]:
df1.sample(3)

In [ ]:
#invoice_date
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y')

#customer_id
df1['customer_id'] = df1['customer_id'].astype(int)

In [ ]:
df1.sample(3)

In [ ]:
df1.dtypes

## Descriptive Statistics

In [ ]:
#here we identify state of variables, but take action just on proper sections ahead.
num_attributes = df1.select_dtypes(include=['int64','float64'])
cat_attributes = df1.select_dtypes(exclude=['int64','float64','datetime64[ns]'])

### Numerical Attributes

In [ ]:
num_attributes.head()

In [ ]:
# central tendency - mean, median
ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T
             
# dispersion - desvio padrão, minimo, maximo, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T
d2 = pd.DataFrame( num_attributes.apply( np.min ) ).T
d3 = pd.DataFrame( num_attributes.apply( np.max ) ).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T

# concatenate
num_metrics = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6] ).T.reset_index()
num_metrics.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
num_metrics

### Cathegorical Attributes

In [ ]:
cat_attributes.head()

#### invoice_no

In [ ]:
# problem: we have invoice_no with letters and numbers
#cat_attributes['invoice_no'].astype( int )  # -> error: ex.'C536379'

# identify number of sales with characters on invoice_no: 
df_letter_invoices = df1.loc[df1['invoice_no'].apply( lambda x: bool( re.search( '[^0-9]+', x ) ) ), :]
df_letter_invoices

In [ ]:
#looks like all invoice_no with C, has negative quantity. Lets check:
print( f'Total number of invoices: {len( df_letter_invoices )}')
print( f'Total number of negative quantity: {len( df_letter_invoices[ df_letter_invoices["quantity"] < 0 ] )}') 
#3 of difference, let's ignore them

#### stock_code

In [ ]:
df1['stock_code']

In [ ]:
# get stock_codes with only letters
df1.loc[df1['stock_code'].apply( lambda x: bool( re.search( '^[a-zA-Z]+$', x ) ) ), 'stock_code'].unique()
#now we have new stock_codes!

In [ ]:
#find samples:
df1.loc[df1['stock_code'].apply( lambda x: bool( re.search( '^[a-zA-Z]+$', x ) ) ) ].sample(10)

# Variable Filtering

In [ ]:
df2 = df1.copy()
#df2.to_csv("../data/interim/cycle8/df2_data_description_done.csv")

In [ ]:
#remove bad users:
df2 = df2.loc[~df2['customer_id'].isin([16446, 15749, 12346]) ]

##Cat Attr

#2. stock_code - remove useless values:
df2 = df2.loc[~df2['stock_code'].isin( ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY','DCGSSGIRL', 'PADS', 'B', 'CRUK'] ) ]
#3. description - remove useless feature:
df2 = df2.drop('description', axis=1)
#4. country - remove 2 values:
df2 = df2[~df2['country'].isin( ['European Community', 'Unspecified' ] ) ]

##Num Attr
#2. unit_price - remove <0.03:
df2 = df2.loc[df2['unit_price'] >= 0.04 ]
#1. quantity - separete into 2 datasets:
df2_returns = df2.loc[df2['quantity'] < 0]
df2_purchases = df2.loc[df2['quantity'] > 0] #there is no quantity == 0.

# Feature Engeneering

In [ ]:
df3 = df2.copy()
df3_returns = df2_returns.copy()
df3_purchases = df2_purchases.copy()
#df3.to_csv("../data/interim/cycle8/df3_var_filtering_done.csv")
#df3_returns.to_csv("../data/interim/cycle8/df3_returns.csv")
#df3_purchases.to_csv("../data/interim/cycle8/df3_purchases.csv")

In [ ]:
#create the referente table with uniques customer_id (and reset index) 
df_ref = df3.drop(['invoice_no', 'stock_code', 'quantity', 'invoice_date',
       'unit_price', 'country'], axis=1).drop_duplicates(ignore_index=True).copy()

In [ ]:
df_ref

## Gross Revenue

In [ ]:
# Gross Revenue: (quantity * price of each purchase)
    #here, we just want to consider gross revenue from sales, not discounting returns, so lets use df3_purchases
df3_purchases['gross_revenue'] = df3_purchases['quantity'] * df3_purchases['unit_price']
df_monetary = df3_purchases[['customer_id','gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge( df_ref, df_monetary, on='customer_id', how='left' )
df_ref.isna().sum()

## Recency

In [ ]:
# Recency: number of days since last purchase
    #here, we want to know the last day this customer bought. If he returned, we don't want to count that day as last purchase, so let's consider just df3_purchases
df_recency = df3_purchases[['customer_id','invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df3_purchases['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_ref = pd.merge( df_ref, df_recency, on='customer_id', how='left' )
df_ref.isna().sum()

## Invoice Quantity

In [ ]:
#quantity of invoices per customer
    #consider just purchases, not returns
df_invoice = df3_purchases[['customer_id','invoice_no']].drop_duplicates().groupby('customer_id').count().reset_index().rename(columns={'invoice_no':"qtt_invoices"})
df_ref = pd.merge( df_ref, df_invoice, on='customer_id', how='left')
df_ref.isna().sum()

## Unique Products

In [ ]:
#quantity of distinct products per customer
    #consider just purchases, not returns
df_invoice = df3_purchases[['customer_id','stock_code']].drop_duplicates().groupby('customer_id').count().reset_index().rename(columns={'stock_code':"unique_products"})
df_ref = pd.merge( df_ref, df_invoice, on='customer_id', how='left')
df_ref.isna().sum()

## Unique Items

In [ ]:
#quantity of items purchased per customer
    #consider just purchases, not returns
df_prod_quantity = df3_purchases[['customer_id','quantity']].groupby('customer_id').sum().reset_index().rename(columns ={'quantity':'unique_items'})
df_ref = pd.merge( df_ref, df_prod_quantity, on='customer_id', how='left')
df_ref.isna().sum()

## Daily Purchase Rate

In [ ]:
# purchase rate per day during the period
#per customer and invoice, get min and max invoice date, total days between min and max, and quantity of invoices
df_aux = ( df3_purchases[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
            .groupby( 'customer_id')
            #new column name ('apply on this columns', 'operation')
            .agg( max_invoice_date = ( 'invoice_date', 'max' ), 
                  min_invoice_date = ( 'invoice_date', 'min' ),
                  invoice_total_days= ( 'invoice_date', lambda x: ((x.max() - x.min()).days)+1),
                  invoice_count = ( 'invoice_no', 'count' ) ) ).reset_index()

# Frequency: invoice_count / invoice_total_days (if invoice_total_days != 0)
df_aux['daily_purchase_rate'] = df_aux[['invoice_count', 'invoice_total_days']].apply( 
    lambda x: x['invoice_count'] / x['invoice_total_days'] 
         if  x['invoice_total_days'] != 0 else 0, axis=1 )

# Merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'daily_purchase_rate']], on='customer_id', how='left' )

df_ref.isna().sum()

## Returns

In [ ]:
#number of products (items) returned per customer
df_returns = df3_returns[['customer_id','quantity']].groupby('customer_id').sum().reset_index().rename( columns={'quantity':'total_prod_returned'} )

#convert to positive, cause we know it's a devolution, no need to be negative anymore:
df_returns['total_prod_returned'] = df_returns['total_prod_returned'] * -1

#bind
df_ref = pd.merge( df_ref, df_returns, how='left', on='customer_id' )

#since df_ref has all customers (purchases + returns), we can say these 4191 with NA in total_prod_returned are from df3_purchases, so let's assign zero to them:
df_ref.loc[df_ref['total_prod_returned'].isna(), 'total_prod_returned'] = 0

df_ref.isna().sum()

# Data Preparation

In [ ]:
df_ref.head()

In [ ]:
df_ref.isna().sum()

In [ ]:
df_ref = df_ref.dropna() #was at EDA
df5 = df_ref.copy() #because EDA Uni and Biv were deleted
df5

In [ ]:
gr_scaller = pickle.load(open("/Users/home/repos/pa005_fidelity_program/src/features/gross_revenue.pkl", "rb") )
#gr_scaller = pickle.load(open("../features/gross_revenue.pkl", "rb") )
df5['gross_revenue'] = gr_scaller.transform(df5[['gross_revenue']])
             
rd_scaller = pickle.load(open('/Users/home/repos/pa005_fidelity_program/src/features/recency_days.pkl', 'rb') )
df5['recency_days'] = rd_scaller.transform( df5[['recency_days']])

# ## df5['qtt_invoices'] = mm.fit_transform(df5[['qtt_invoices']])  #feature not selected 

up_scaller = pickle.load(open('/Users/home/repos/pa005_fidelity_program/src/features/unique_products.pkl', 'rb') )
df5['unique_products'] = up_scaller.transform(df5[['unique_products']])

# ## df5['unique_items'] = mm.fit_transform(df5[['unique_items']])  #feature not selected 

dpr_scaller=pickle.load(open('/Users/home/repos/pa005_fidelity_program/src/features/daily_purchase_rate.pkl', 'rb') )
df5['daily_purchase_rate'] = dpr_scaller.transform(df5[['daily_purchase_rate']])

tpr_scaller=pickle.load(open('/Users/home/repos/pa005_fidelity_program/src/features/total_prod_returned.pkl', 'rb') )
df5['total_prod_returned']=tpr_scaller.transform(df5[['total_prod_returned']])

In [ ]:
#after transformations:
df5.head()

# Feature Selection

In [ ]:
df6 = df5.copy()
#df6.to_csv("../data/interim/cycle8/df6_data_prep_done.csv")

In [ ]:
#select features for variable space

#drop customer_id
X = df6.drop('customer_id', axis=1).copy()
X.head()

In [ ]:
#select features
selected_features = ['gross_revenue','recency_days','unique_products','daily_purchase_rate','total_prod_returned'] #5

X = X[selected_features].copy()
X.head()

# EDA - Data Space Study

In [ ]:
#as soon as data on original space is not organized, let's look for a better data space with embedding
X.head()

In [ ]:
#keep just Umap embedding space (the best)

## UMAP

In [ ]:
#UMAP basically takes a dataset in a very high dimension and returns a new dataset with the same number of samples in a space of much lower dimension than the original dimension.
#UMAP is good for a lot of data because it's faster, but it's not as verbose in visualization.

In [ ]:
#reducer = umap.UMAP( random_state=42 )
reducer = pickle.load(open('/Users/home/repos/pa005_fidelity_program/src/features/umap_model.pkl','rb'))
embedding = reducer.transform( X ) #now just transform

df_umap = pd.DataFrame()
df_umap['embedding_x'] = embedding[:, 0]
df_umap['embedding_y'] = embedding[:, 1]
sns.scatterplot( x='embedding_x', y='embedding_y', data=df_umap )

In [ ]:
#evaluation, passing df_umap as dataframe

#set number of clusters
clusters_em = np.arange(2,13,1)
clusters_em

#let's use scipy instead of sklearn, being a simpler implementation.
hc_list_em = []
for k in clusters_em:
    #model definition and training
    hc_model_em = hc.linkage(df_umap, 'ward')

    #model predict
    hc_labels_em = hc.fcluster(hc_model_em, k, criterion='maxclust')#to cut dendrogram

    #model perfomance (SS)
    hc_ss_em = m.silhouette_score(df_umap, hc_labels_em, metric='euclidean')
    hc_list_em.append(hc_ss_em) 

In [ ]:
#Silhouette Score (with tree-based embedding from not scaled dataset)
plt.plot(clusters_em, hc_list_em, linestyle='--', marker='o', color='b')
plt.xlabel('K');
plt.ylabel('Silhouette Score');
plt.title('Silhouette Score x K');

# Model Training

In [ ]:
#let's keep using Umap embedding space, where on HC with 8k, we've got an SS of 0.55 
#8Ks is also a good number of cluster for business team handle actions from them
X = df_umap
X

In [ ]:
#8 clusters, as defined
k = 8

#let's use scipy instead of sklearn, being a simpler implementation.
#model definition and training
hc_model = hc.linkage(X, 'ward')

#model predict
hc_labels = hc.fcluster(hc_model, k, criterion='maxclust')#to cut dendrogram

#model perfomance (SS)
hc_ss = m.silhouette_score(X, hc_labels, metric='euclidean') 
print(f'SS Value for {k} clusters: {hc_ss}')

# Cluster Analysis

## Cluster Profile

In [ ]:
df_ref

In [ ]:
df9p = df_ref.copy() #df_ref contains original values (not reescaled), to use in cluster profile
ids_and_vars_selected = ['customer_id','gross_revenue','recency_days','unique_products','daily_purchase_rate','total_prod_returned']
df9p = df9p[ids_and_vars_selected]
df9p['cluster'] = hc_labels

#change dtypes
df9p['recency_days'] = df9p['recency_days'].astype(int)
df9p['unique_products'] = df9p['unique_products'].astype(int)
df9p['total_prod_returned'] = df9p['total_prod_returned'].astype(int)

#record timestamp of training
df9p['last_training'] = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df9p

In [ ]:
#building df_cluster

# Number of customer
df_cluster = df9p[['customer_id', 'cluster']].groupby('cluster').count().reset_index()
df_cluster['perc_customer'] = df_cluster['customer_id'] / df_cluster['customer_id'].sum()*100

# Avg gross revenue
df_avg_gross_revenue = df9p[['gross_revenue','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_gross_revenue, how='inner', on='cluster')

# Avg recency days
df_avg_recency_days = df9p[['recency_days','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_recency_days, how='inner', on='cluster')

# Avg unique products
df_avg_unique_products = df9p[['unique_products','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_unique_products, how='inner', on='cluster')

# Avg daily purchase rate
df_avg_daily_purchase_rate = df9p[['daily_purchase_rate','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_daily_purchase_rate, how='inner', on='cluster')

# Avg total products returned
df_avg_total_prod_returned = df9p[['total_prod_returned','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_total_prod_returned, how='inner', on='cluster')

# cluster profiles
df_cluster.sort_values('gross_revenue', ascending=False)

In [ ]:
#df_cluster.to_csv('../data/interim/cycle8/df_cluster_kmeans_8k.csv')

In [ ]:
#generate the repport dinamically:
for i in range(len(df_cluster['customer_id'])):
    print(f""" Cluster {df_cluster['cluster'][i]}:
    -Number of customers: {df_cluster['customer_id'][i]} ({round(df_cluster['perc_customer'][i])}%)
    -Average revenue: ${round(df_cluster['gross_revenue'][i])}
    -Average recency: by each {round(df_cluster['recency_days'][i])} days ({round(df_cluster['recency_days'][i]/7)} week(s))
    -Average unique products purchased: {round(df_cluster['unique_products'][i])}  
    -Average purchases/month: {round((df_cluster['daily_purchase_rate'][i])*30 ,1)} 
    -Average total products returned: {round(df_cluster['total_prod_returned'][i])} """)

# Deploy

## Insert into SQLITE

In [ ]:
df9p

In [ ]:
df9p.dtypes

In [ ]:
#create db connection (and db_file if sqlite)

##sqlite
#endpoint = 'sqlite:////Users/home/repos/pa005_fidelity_program/notebooks/loyals_db.sqlite' 

##postgre 
load_dotenv() #load from .env
pg_user = os.environ['pg_user'] 
pg_passwd = os.environ['pg_passwd'] 
pg_host = os.environ['pg_host'] 
pg_port = os.environ['pg_port'] 
#build endpoint
endpoint = f'postgresql://{pg_user}:{pg_passwd}@{pg_host}:{pg_port}'
        
#create connection
db = create_engine(endpoint, poolclass=NullPool)
conn = db.connect()

In [ ]:
# #check if table exists on sqlite
# check_table = """
#     SELECT name FROM sqlite_master WHERE type='table' AND name='loyals';
# """
# df_check = pd.read_sql_query(check_table, conn)

# #create table if does not exist
# if len(df_check) == 0:  #0 = table does not exist, 1 = table exists
#     query_create_table_loyals = """
#     CREATE TABLE loyals (
#         customer_id              INTEGER,
#         gross_revenue            REAL,
#         recency_days             INTEGER,
#         unique_products          INTEGER,
#         daily_purchase_rate      REAL,
#         total_prod_returned      INTEGER,
#         cluster                  INTEGER,
#         last_training            TEXT
#         )"""
#     conn.execute( query_create_table_loyals )
#     print('Table loyals was created!')
# else:
#     print('Table loyals exists!')

In [ ]:
#create table
# query_create_table_loyals = """
#     CREATE TABLE loyals (
#         customer_id              INTEGER,
#         gross_revenue            REAL,
#         recency_days             INTEGER,
#         unique_products          INTEGER,
#         daily_purchase_rate      REAL,
#         total_prod_returned      INTEGER,
#         cluster                  INTEGER,
#         last_training            TEXT
#         )"""
# conn.execute( query_create_table_loyals )

In [ ]:
#insert data into table loyals using sqlalchemy, appending data
df9p.to_sql('loyals', con=conn, if_exists='append', index=False )#index=False to ignore dataframe index

In [ ]:
#consult database
query = """
    SELECT * FROM loyals
"""
df = pd.read_sql_query(query, conn)
df

In [ ]:
conn.close() #closes connection